In [3]:
import pandas as pd
import os

# --- 1. CONFIGURAZIONE ---
DATA_DIR = '../dataset'
# Carichiamo il file su cui abbiamo ricalcolato il Target
INPUT_FILE = os.path.join(DATA_DIR, 'VAERS_FIXED_NO_RECOVD.csv')
OUTPUT_FILE = os.path.join(DATA_DIR, 'VAERS_READY_FOR_SPLIT.csv')

print(f"Caricamento file: {INPUT_FILE}...")
df = pd.read_csv(INPUT_FILE, low_memory=False)

# --- 2. LISTA DELLE COLONNE DA ELIMINARE ---
# Ho unito le amministrative, le ridondanti e quelle di esito medico (Leakage)
cols_to_drop = [
    # Amministrative / Rumore
    'RECVDATE', 'STATE', 'RPT_DATE', 'V_ADMINBY', 'V_FUNDBY',
    'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE', 'VAX_LOT',

    # Ridondanti (Età e Vaccino già coperti)
    'CAGE_YR', 'CAGE_MO', 'VAX_NAME', 'VAX_SITE', 'VAX_ROUTE',

    # Esito Medico / Data Leakage (Abbiamo già creato IS_SEVERE)
    'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL',
    'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'BIRTH_DEFECT',
    'OFC_VISIT', 'ER_ED_VISIT'
]

# --- 3. ESECUZIONE ELIMINAZIONE ---
# Filtriamo la lista per eliminare solo quelle che esistono effettivamente nel file
existing_drops = [c for c in cols_to_drop if c in df.columns]

print(f"Eliminazione di {len(existing_drops)} colonne in corso...")
df_final = df.drop(columns=existing_drops)

# --- 4. VERIFICA E SALVATAGGIO ---
print("\n--- DATASET RISULTANTE ---")
print(f"Colonne totali rimaste: {len(df_final.columns)}")
print(f"Colonne mantenute: {df_final.columns.tolist()}")

df_final.to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ Successo! Il file pulito è stato salvato in: {OUTPUT_FILE}")

# Check veloce sul target
if 'IS_SEVERE' in df_final.columns:
    print(f"Il Target 'IS_SEVERE' è presente. Distribuzione:\n{df_final['IS_SEVERE'].value_counts()}")

Caricamento file: data/VAERS_FIXED_NO_RECOVD.csv...
Eliminazione di 10 colonne in corso...

--- DATASET RISULTANTE ---
Colonne totali rimaste: 19
Colonne mantenute: ['VAERS_ID', 'AGE_YRS', 'SEX', 'SYMPTOM_TEXT', 'VAX_DATE', 'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'ALLERGIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_DOSE_SERIES', 'NUMERO_SINTOMI', 'LISTA_SINTOMI', 'IS_SEVERE']

✅ Successo! Il file pulito è stato salvato in: data/raw/VAERS_READY_FOR_SPLIT.csv
Il Target 'IS_SEVERE' è presente. Distribuzione:
IS_SEVERE
0    797523
1    112085
Name: count, dtype: int64


In [4]:
import pandas as pd
import os

# --- CONFIGURAZIONE ---
DATA_DIR = '../dataset'
INPUT_FILE = os.path.join(DATA_DIR, 'VAERS_READY_FOR_SPLIT.csv')
OUTPUT_FILE = os.path.join(DATA_DIR, 'VAERS_FINAL_FOR_ML.csv')

df = pd.read_csv(INPUT_FILE, low_memory=False)

# 1. Eliminiamo le date (teniamo NUMDAYS che è il dato utile)
# 2. Eliminiamo SYMPTOM_TEXT perché hai già LISTA_SINTOMI (più pulita)
cols_to_drop = ['VAX_DATE', 'ONSET_DATE', 'SYMPTOM_TEXT','VAX_TYPE','LAB_DATA','PRIOR_VAX','VAX_TYPE']

# Eliminiamo solo se presenti
df_final = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

print(f"Dataset finale pronto: {len(df_final.columns)} colonne.")
df_final.to_csv(OUTPUT_FILE, index=False)

Dataset finale pronto: 13 colonne.
